# <centre>Convolve Epoch 2 PS1 </centre>

In [241]:

from IPython.display import Image
Image(filename='C:\\Users\\NAVIN PATWARI\\Desktop\\tree.png',width = 300,height =300 ) 

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\NAVIN PATWARI\\Desktop\\tree.png'

In [ ]:
import pandas as pd
from sklearn import model_selection, preprocessing, linear_model, metrics
import numpy as np
from sklearn.metrics import f1_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

## Loading Data

In [212]:
dataset=pd.read_csv("PS1.csv")

In [315]:
dataset.head()

,Unnamed: 0,Business Unit,Product Family,PLID,Fiscal Quarter,Fiscal Month,Booked_Qty,Booking_Date,month,Quarter,QYear,Byear,mean_booked_qty
0,0,CSWBU,CBR8,CBR-D121-DS-MOD,Q1 FY2022,AUG FY2022,204,01-08-2016,AUG,Q1,2022,2016,1959
1,1,CSWBU,CBR8,CBR-D121-DS-MOD,Q1 FY2022,OCT FY2022,564,01-10-2016,OCT,Q1,2022,2016,1959
2,2,CSWBU,CBR8,CBR-D121-DS-MOD,Q1 FY2022,SEP FY2022,1476,01-09-2016,SEP,Q1,2022,2016,1959
3,3,CSWBU,CBR8,CBR-D121-DS-MOD,Q1 FY2002,AUG FY2002,374,01-08-2017,AUG,Q1,2002,2017,2061
4,4,CSWBU,CBR8,CBR-D121-DS-MOD,Q1 FY2002,OCT FY2002,6094,01-10-2017,OCT,Q1,2002,2017,2061


In [317]:
max(dataset['Booked_Qty'])

322807

## Feature Engineering

In [214]:
dataset['mean_booked_qty'] = None
a = dataset.groupby(['PLID','Byear']).agg({'Booked_Qty' : 'mean'})['Booked_Qty']
b = dataset.groupby(['PLID','Byear']).agg({'Booked_Qty' : 'mean'}).index
for i in range(0,len(b)):
    dataset['mean_booked_qty'][(dataset['PLID'] == b[i][0])  & (dataset['Byear'] == b[i][1]) ] = a[i]
dataset['mean_booked_qty'] = dataset['mean_booked_qty'].astype('int64')

C:\Users\ANKITA~1\AppData\Local\Temp/ipykernel_15644/910815301.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['mean_booked_qty'][(dataset['PLID'] == b[i][0])  & (dataset['Byear'] == b[i][1]) ] = a[i]


In [215]:
plid=dataset["PLID"].unique()

In [216]:
dataset["Product Family"].nunique()

73

In [217]:
dataset["Business Unit"].nunique()

18

In [218]:
dataset.describe

<bound method NDFrame.describe of        Unnamed: 0 Business Unit Product Family             PLID  \
0               0         CSWBU           CBR8  CBR-D121-DS-MOD   
1               1         CSWBU           CBR8  CBR-D121-DS-MOD   
2               2         CSWBU           CBR8  CBR-D121-DS-MOD   
3               3         CSWBU           CBR8  CBR-D121-DS-MOD   
4               4         CSWBU           CBR8  CBR-D121-DS-MOD   
...           ...           ...            ...              ...   
10991       10991         RTOBU        C980040      C9800-40-K9   
10992       10992         RTOBU        C980040      C9800-40-K9   
10993       10993         RTOBU        C980040      C9800-40-K9   
10994       10994         RTOBU        C980040      C9800-40-K9   
10995       10995         RTOBU        C980040      C9800-40-K9   

      Fiscal Quarter Fiscal Month  Booked_Qty Booking_Date month Quarter  \
0          Q1 FY2022   AUG FY2022         204   01-08-2016   AUG      Q1   
1        

In [219]:
data=dataset.drop(["Fiscal Quarter","Fiscal Month","Booking_Date"],axis="columns")

In [220]:
data["Business Unit"].value_counts()

UOPBLRBU    1675
SRSBU       1392
CSA         1271
TOASTBU     1012
UCEBU        917
QKBU         732
CRBU         725
BATBU        615
OTBU         499
ERBU         488
CVEBU        426
EBBU         343
RTOBU        259
ECBU         204
CSWBU        151
IOTBU        149
NGFWBU        69
CPBU          69
Name: Business Unit, dtype: int64

In [221]:
cleanup_nums = {"month":     {"JAN": 1, "FEB": 2,"MAR": 3,
                                      "APR":4,"MAY":5,"JUN":6,"JUL":7,
                                    "AUG":8,"SEP":9,"OCT":10,"NOV":11,
                                 "DEC":12},
                "Quarter": {"Q1": 1, "Q2": 2,"Q3":3,"Q4":4},
               "Business Unit": {"UOPBLRBU":111,"SRSBU":112,"CSA":113,"TOASTBU":114,"UCEBU":115,"QKBU":116,
                                "CRBU":117,"BATBU":118,"OTBU":119,"ERBU":120,"CVEBU":121,"EBBU":122,"RTOBU":123,"ECBU":124,
                                 "CSWBU":125,"IOTBU":126,"NGFWBU":127,"CPBU":128}}

In [222]:
data = data.replace(cleanup_nums)

data.head()

,Unnamed: 0,Business Unit,Product Family,PLID,Booked_Qty,month,Quarter,QYear,Byear,mean_booked_qty
0,0,125,CBR8,CBR-D121-DS-MOD,204,8,1,2022,2016,1959
1,1,125,CBR8,CBR-D121-DS-MOD,564,10,1,2022,2016,1959
2,2,125,CBR8,CBR-D121-DS-MOD,1476,9,1,2022,2016,1959
3,3,125,CBR8,CBR-D121-DS-MOD,374,8,1,2002,2017,2061
4,4,125,CBR8,CBR-D121-DS-MOD,6094,10,1,2002,2017,2061


In [223]:
data.sort_values(by=['Byear','PLID', 'Quarter','month'],inplace=True)

In [224]:
 data[data["PLID"]=="A9K1000-ESP100"]["Booked_Qty"]

6218     10
6220     16
6219    123
6249     63
6250     34
       ... 
6247     52
6246    204
6276     76
6277    115
6275     47
Name: Booked_Qty, Length: 114, dtype: int64

In [225]:
Booked_Qty = data['Booked_Qty'].astype ('float')
qty_lag = []
for i in range (0,len(Booked_Qty)-1):
        k =Booked_Qty[i] 
        qty_lag.append(k)

In [226]:
# qty_lag
qty_lag.insert(0,0)


In [227]:
data["qty_lag"]=None
data["qty_lag"]=qty_lag

In [228]:
# print(data.groupby(['Product Family']).\
#       agg({'Target' : 'mean'}))

# df['lag(a,1)'] = df['a'].shift(1)
num = [col for col in data.columns if data[col].dtypes != 'O']
data[num].head()
data[num].corr()

,Unnamed: 0,Business Unit,Booked_Qty,month,Quarter,QYear,Byear,mean_booked_qty,qty_lag
Unnamed: 0,1.000000,-0.393810,0.354880,-0.010582,0.008924,-0.050395,-0.162401,0.396439,-0.099925
Business Unit,-0.393810,1.000000,-0.061875,-0.001289,0.002673,0.001417,0.030037,-0.069174,0.140956
Booked_Qty,0.354880,-0.061875,1.000000,-0.003177,0.019072,0.011387,-0.089175,0.894477,0.007537
month,-0.010582,-0.001289,-0.003177,1.000000,-0.441151,0.152788,0.117132,0.011663,0.087639
Quarter,0.008924,0.002673,0.019072,-0.441151,1.000000,-0.253114,-0.200477,-0.027656,-0.148543
QYear,-0.050395,0.001417,0.011387,0.152788,-0.253114,1.000000,0.372507,0.012731,0.318401
Byear,-0.162401,0.030037,-0.089175,0.117132,-0.200477,0.372507,1.000000,-0.099696,0.307849
mean_booked_qty,0.396439,-0.069174,0.894477,0.011663,-0.027656,0.012731,-0.099696,1.000000,0.010693
qty_lag,-0.099925,0.140956,0.007537,0.087639,-0.148543,0.318401,0.307849,0.010693,1.000000


In [229]:
# density_map = data.groupby(['Product Family']).\
#     agg({'Target' : 'mean'}).to_dict()['Target']
# new_data = data.copy()
# new_data['Product Family'] = new_data['Product Family'].map(density_map)
# print(new_data)

In [230]:
# new_data["Product Family Type Count"].nunique()

In [231]:
datas=data.drop(["Booked_Qty","Unnamed: 0"],axis="columns")
Y=data["Booked_Qty"]

In [232]:
datas.head()

,Business Unit,Product Family,PLID,month,Quarter,QYear,Byear,mean_booked_qty,qty_lag
6218,112,A9K1000,A9K1000-ESP100,8,1,2013,2013,62,0.0
6220,112,A9K1000,A9K1000-ESP100,9,1,2013,2013,62,204.0
6219,112,A9K1000,A9K1000-ESP100,10,1,2013,2013,62,564.0
6249,112,A9K1000,A9K1000-ESP100,1,2,2013,2013,62,1476.0
6250,112,A9K1000,A9K1000-ESP100,11,2,2013,2013,62,374.0


In [233]:
X = pd.get_dummies(datas, drop_first=True)
# from sklearn.datasets import make_regression
# X, Y = make_regression(random_state=11)

## Splitting Data for Training and Validation

In [234]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)

## Model Implementation

In [235]:
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=5)
from sklearn.model_selection import cross_val_score

### Linear Regression

In [236]:
from sklearn import linear_model
lin=linear_model.LinearRegression()
lasso_reg = linear_model.Lasso(alpha=100, max_iter=100, tol=0.1)
lasso_reg.fit(X_train, y_train)
lasso_reg.score(X_test,y_test)

0.8082272185930683

### Ridge Regression

In [237]:
from sklearn.linear_model import Ridge
ridge_reg= Ridge(alpha=10, max_iter=100, tol=0.3)
ridge_reg.fit(X_train, y_train)
ridge_reg.score(X_test,y_test)

0.802063123190295

### Gradient Boosting

In [238]:
from sklearn.ensemble import GradientBoostingRegressor
cl = GradientBoostingRegressor(n_estimators=500, learning_rate=0.10, max_depth=10,random_state=11)
cl.fit(X_train, y_train)
cl.score(X_test,y_test)

0.8480729555658475

### LightGBM

In [239]:
!pip install lightgbm
from lightgbm import LGBMRegressor

lgbm = LGBMRegressor(random_state=5)

lgbm.fit(X_train, y_train)
lgbm.score(X_test,y_test)

y_pred = lgbm.predict(X_test)

In [240]:
lgbm = LGBMRegressor(random_state=5)

lgbm.fit(X_train, y_train)
lgbm.score(X_test,y_test)

# y_pred = lgbm.predict(X_test)

0.8940370078691334

In [151]:
rmse = np.sqrt(MSE(y_test, y_pred))
print("RMSE : % f" %(rmse))

RMSE :  6305.473061


### Lasso Regression

In [152]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1,max_iter=10000)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.8202441727544199

### Cross Validation

In [153]:
cross_val_score(cl,X,Y)

array([0.7992293 , 0.56317093, 0.83743235, 0.8083865 , 0.77486495])

In [113]:
from sklearn.metrics import mean_squared_error as MSE

In [157]:
train_dmatrix = xg.DMatrix(data = X_train, label = y_train)
test_dmatrix = xg.DMatrix(data = X_test, label = y_test)
 
# Parameter dictionary specifying base learner
param = {"booster":"gblinear", "objective":"reg:linear"}
 
xgb_r = xg.train(params = param, dtrain = train_dmatrix, num_boost_round = 20)
pred = xgb_r.predict(test_dmatrix)
 
# RMSE Computation
rmse = np.sqrt(MSE(y_test, pred))
print("RMSE : % f" %(rmse))

[15:46:25] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-030221e36e1a46bfb-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
RMSE :  8118.883744


### Enhancing Predictions

In [166]:
pred1 = []
for i in pred:
    if i<0:
        pred1.append(0)
    else:
        pred1.append(i)
pred = pred1

### Extracting Data for Comparision between predicted and actual value

In [176]:
X_test[X_test['PLID_UCSB-B200-M5'] == 1]
df = X_train[(X_train['PLID_UCS-S3260-HDW18T'] == 1) & (X_train['Byear'] == 2022)]
df_pred = lgbm.predict(df)
df_has = [y_train[1784],y_train[1783],y_train[1782],y_train[1781],y_train[1780],y_train[1779]]
df = pd.DataFrame(df['month'])
df['has'] = df_has
df['pred'] = df_pred
df.to_csv('data_compare.csv')

,Business Unit,month,Quarter,QYear,Byear,mean_booked_qty,qty_lag,Product Family_4300ISR,Product Family_8000,Product Family_A9K1000,...,PLID_UCS-S3260-HDW18T,PLID_UCSB-B200-M5,PLID_UCSC-C220-M5SX,PLID_UCSC-C240-M5SX,PLID_UCSC-C240-M6SX,PLID_UCSC-GPU-T4-16,PLID_UCSX-210C-M6,PLID_VEDGE-2000-AC-K9,PLID_WS-C3560CX-12PC-S,PLID_WS-C3650-48FQM
1090,113,12,2,2022,2022,5479,6366.0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1109,113,5,4,2008,2018,4002,269.0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1096,113,4,3,2007,2019,7848,450.0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1100,113,2,3,2011,2020,7542,232.0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [249]:
ddata = data.reset_index()

In [252]:
ddata = ddata.drop(['index','Unnamed: 0'],axis = 1)

In [257]:
new_data = ddata[((ddata['Quarter'] == 3) | (ddata['Quarter'] == 4)) & (ddata['Byear'] == 2021)]

In [260]:
new_data['Byear'] = 2022
new_data['QYear'] = 2022

C:\Users\ANKITA~1\AppData\Local\Temp/ipykernel_15644/3295378516.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Byear'] = 2022
C:\Users\ANKITA~1\AppData\Local\Temp/ipykernel_15644/3295378516.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['QYear'] = 2022


In [262]:
new_data = new_data.drop(['Booked_Qty','Qyear'], axis = 1)
new_data.head()

,Business Unit,Product Family,PLID,month,Quarter,QYear,Byear,mean_booked_qty,qty_lag
6721,117,8000,8201-SYS,2,3,2022,2022,25,5371.0
6722,117,8000,8201-SYS,3,3,2022,2022,25,4399.0
6723,117,8000,8201-SYS,4,3,2022,2022,25,7963.0
6724,117,8000,8201-SYS,5,4,2022,2022,25,3393.0
6725,117,8000,8201-SYS,6,4,2022,2022,25,6247.0


In [283]:
a = list(ddata[(ddata['month'] == 1) & (ddata['Byear'] == 2022 )]['PLID'])
b = list(ddata[(ddata['month'] == 1) & (ddata['Byear'] == 2022 )]['Booked_Qty'])

In [284]:
for i in range (0,len(a)):
    new_data['qty_lag'][(new_data['month'] == 2) & (new_data['PLID'] == a[i])] = b[i]

C:\Users\ANKITA~1\AppData\Local\Temp/ipykernel_15644/1626229019.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['qty_lag'][(new_data['month'] == 2) & (new_data['PLID'] == a[i])] = b[i]


In [285]:
new_data[new_data['month'] == 2]

,Business Unit,Product Family,PLID,month,Quarter,QYear,Byear,mean_booked_qty,qty_lag
6721,117,8000,8201-SYS,2,3,2022,2022,25,505.0
6738,117,8000,8800-LC-36FH,2,3,2022,2022,24,456.0
6750,117,8000,8800-LC-48H,2,3,2022,2022,472,428.0
6762,117,8000,8808-FC,2,3,2022,2022,596,240.0
6774,117,8000,8808-SYS,2,3,2022,2022,136,141.0
...,...,...,...,...,...,...,...,...,...
9437,113,UCSC,UCSC-C240-M5SX,2,3,2022,2022,1026,506.0
9453,113,UCST,UCSC-GPU-T4-16,2,3,2022,2022,396,332.0
9470,112,VIPTELA,VEDGE-2000-AC-K9,2,3,2022,2022,256,109.0
9482,111,C3560CX,WS-C3560CX-12PC-S,2,3,2022,2022,3719,2713.0


In [290]:
x2 = pd.get_dummies(new_data[new_data['month'] == 2], drop_first=True)
a = X_test.columns
b = x2.columns
missing = []
for i in a:
    if i not in b:
        missing.append(i)

In [291]:
x2[missing] = 0

In [293]:
y2 = lgbm.predict(x2)

In [297]:
new_data['pred_value'] = None
new_data['pred_value'][new_data['month'] == 2] = y2

C:\Users\ANKITA~1\AppData\Local\Temp/ipykernel_15644/3303001385.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['pred_value'][new_data['month'] == 2] = y2


In [299]:
new_data[new_data['month'] == 2]

,Business Unit,Product Family,PLID,month,Quarter,QYear,Byear,mean_booked_qty,qty_lag,pred_value
6721,117,8000,8201-SYS,2,3,2022,2022,25,505.0,172.300826
6738,117,8000,8800-LC-36FH,2,3,2022,2022,24,456.0,292.366495
6750,117,8000,8800-LC-48H,2,3,2022,2022,472,428.0,570.528357
6762,117,8000,8808-FC,2,3,2022,2022,596,240.0,678.501057
6774,117,8000,8808-SYS,2,3,2022,2022,136,141.0,227.524203
...,...,...,...,...,...,...,...,...,...,...
9437,113,UCSC,UCSC-C240-M5SX,2,3,2022,2022,1026,506.0,1121.607091
9453,113,UCST,UCSC-GPU-T4-16,2,3,2022,2022,396,332.0,515.510409
9470,112,VIPTELA,VEDGE-2000-AC-K9,2,3,2022,2022,256,109.0,346.507358
9482,111,C3560CX,WS-C3560CX-12PC-S,2,3,2022,2022,3719,2713.0,3982.691887


#### Predict March

In [304]:
a = list(new_data[(new_data['month'] == 3) & (new_data['Byear'] == 2022 )]['PLID'])
b = list(new_data[(new_data['month'] == 3) & (new_data['Byear'] == 2022 )]['pred_value'])
for i in range (0,len(a)):
    new_data['qty_lag'][(new_data['month'] == 3) & (new_data['PLID'] == a[i])] = b[i]
fake_data = new_data.drop(['pred_value'],axis = 1)
x3 = pd.get_dummies(new_data[fake_data['month'] == 3], drop_first=True)
a = X_test.columns
b = x3.columns
df_for_pred = pd.DataFrame()
for i in a:
    if i in b:
        df_for_pred[i] = x3[i]
    else:
        df_for_pred[i] = 0
y3 = lgbm.predict(df_for_pred)
new_data['pred_value'][new_data['month'] == 3] = y3

C:\Users\ANKITA~1\AppData\Local\Temp/ipykernel_15644/2123103144.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['qty_lag'][(new_data['month'] == 3) & (new_data['PLID'] == a[i])] = b[i]
C:\Users\ANKITA~1\AppData\Local\Temp/ipykernel_15644/2123103144.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_for_pred[i] = x3[i]
C:\Users\ANKITA~1\AppData\Local\Temp/ipykernel_15644/2123103144.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all co

#### function to predict the remaining months

In [305]:
def pred(month):
    a = list(new_data[(new_data['month'] == month) & (new_data['Byear'] == 2022 )]['PLID'])
    b = list(new_data[(new_data['month'] == month) & (new_data['Byear'] == 2022 )]['pred_value'])
    for i in range (0,len(a)):
        new_data['qty_lag'][(new_data['month'] == month) & (new_data['PLID'] == a[i])] = b[i]
    fake_data = new_data.drop(['pred_value'],axis = 1)
    xn = pd.get_dummies(new_data[fake_data['month'] == month], drop_first=True)
    a = X_test.columns
    b = xn.columns
    df_for_pred = pd.DataFrame()
    for i in a:
        if i in b:
            df_for_pred[i] = xn[i]
        else:
            df_for_pred[i] = 0
    yn = lgbm.predict(df_for_pred)
    new_data['pred_value'][new_data['month'] == month] = yn

In [323]:
for i in range(4,8):
    pred(i)

C:\Users\ANKITA~1\AppData\Local\Temp/ipykernel_15644/3967256691.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['qty_lag'][(new_data['month'] == month) & (new_data['PLID'] == a[i])] = b[i]
C:\Users\ANKITA~1\AppData\Local\Temp/ipykernel_15644/3967256691.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_for_pred[i] = xn[i]
C:\Users\ANKITA~1\AppData\Local\Temp/ipykernel_15644/3967256691.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining al

In [331]:
conditions = [(new_data['pred_value'] < 65000),
              ((new_data['pred_value']>= 65000) & (new_data['pred_value'] < 130000)),
             ((new_data['pred_value'] >= 130000) & (new_data['pred_value'] < 195000)),
             ((new_data['pred_value'] >= 195000) & (new_data['pred_value'] < 260000)),
             (new_data['pred_value'] >= 260000)]
values = ['XS','S','M','L','XL']
new_data['inventory'] = None
new_data['inventory'] = np.select(conditions,values)

In [333]:
new_data.to_csv('Output.csv')

In [334]:
new_data

,Business Unit,Product Family,PLID,month,Quarter,QYear,Byear,mean_booked_qty,qty_lag,pred_value,inventory
6721,117,8000,8201-SYS,2,3,2022,2022,25,505.000000,172.300826,XS
6722,117,8000,8201-SYS,3,3,2022,2022,25,156.449034,156.449034,XS
6723,117,8000,8201-SYS,4,3,2022,2022,25,177.470919,177.470919,XS
6724,117,8000,8201-SYS,5,4,2022,2022,25,4.230327,10.551568,XS
6725,117,8000,8201-SYS,6,4,2022,2022,25,102.490155,130.060204,XS
...,...,...,...,...,...,...,...,...,...,...,...
9495,111,C3650,WS-C3650-48FQM,3,3,2022,2022,216,823.760328,313.540732,XS
9496,111,C3650,WS-C3650-48FQM,4,3,2022,2022,216,420.920947,420.920947,XS
9497,111,C3650,WS-C3650-48FQM,5,4,2022,2022,216,158.998640,180.930018,XS
9498,111,C3650,WS-C3650-48FQM,6,4,2022,2022,216,280.837899,302.769277,XS
